## Udacity Data Engineering Capstone Project 

### Setup

In [1]:
import boto3
import pandas as pd
from pathlib import Path
from configparser import ConfigParser

In [2]:
root_path = Path().home().joinpath("/home/vineeth/Work/Extras/Capstone Project/")
data_path = root_path.joinpath("data")

In [3]:
config = ConfigParser()
config.read(root_path.joinpath("config.cfg"))

['/home/vineeth/Work/Extras/Capstone Project/config.cfg']

In [6]:
s3 = boto3.client("s3", aws_access_key_id=config.get("AWS", "ACCESS_KEY_ID"), 
                  aws_secret_access_key=config.get("AWS", "SECRET_ACCESS_KEY"))


s3.download_file(config.get("kinesis", "delivery_stream"), <STREAM>, str(data_path.joinpath("tweets.json")))

In [9]:
happiness_df = pd.read_csv(data_path.joinpath("happiness_index.csv"))
temp_df = pd.read_csv(data_path.joinpath("temperature_city.csv"))
tweet_df = pd.read_json(data_path.joinpath("tweets.json"), orient="records")

### Exploration

In [10]:
happiness_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 12 columns):
Country                          155 non-null object
Happiness.Rank                   155 non-null int64
Happiness.Score                  155 non-null float64
Whisker.high                     155 non-null float64
Whisker.low                      155 non-null float64
Economy..GDP.per.Capita.         155 non-null float64
Family                           155 non-null float64
Health..Life.Expectancy.         155 non-null float64
Freedom                          155 non-null float64
Generosity                       155 non-null float64
Trust..Government.Corruption.    155 non-null float64
Dystopia.Residual                155 non-null float64
dtypes: float64(10), int64(1), object(1)
memory usage: 14.7+ KB


In [11]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8599212 entries, 0 to 8599211
Data columns (total 7 columns):
dt                               object
AverageTemperature               float64
AverageTemperatureUncertainty    float64
City                             object
Country                          object
Latitude                         object
Longitude                        object
dtypes: float64(2), object(5)
memory usage: 459.2+ MB


In [12]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1211 entries, 0 to 1210
Data columns (total 16 columns):
user_id            1211 non-null int64
name               1211 non-null object
nickname           1211 non-null object
description        1211 non-null object
user_location      1211 non-null object
followers_count    1211 non-null int64
tweets_count       1211 non-null int64
user_date          1211 non-null object
verified           1211 non-null bool
tweet_id           1211 non-null int64
text               1211 non-null object
favs               1211 non-null int64
retweets           1211 non-null int64
tweet_date         1211 non-null object
tweet_location     35 non-null object
source             1211 non-null object
dtypes: bool(1), int64(6), object(9)
memory usage: 143.2+ KB


#### Missing values

In [13]:
happiness_df.isnull().any()

Country                          False
Happiness.Rank                   False
Happiness.Score                  False
Whisker.high                     False
Whisker.low                      False
Economy..GDP.per.Capita.         False
Family                           False
Health..Life.Expectancy.         False
Freedom                          False
Generosity                       False
Trust..Government.Corruption.    False
Dystopia.Residual                False
dtype: bool

In [14]:
temp_df.isnull().any()

dt                               False
AverageTemperature                True
AverageTemperatureUncertainty     True
City                             False
Country                          False
Latitude                         False
Longitude                        False
dtype: bool

In [15]:
tweet_df.isnull().any()

user_id            False
name               False
nickname           False
description        False
user_location      False
followers_count    False
tweets_count       False
user_date          False
verified           False
tweet_id           False
text               False
favs               False
retweets           False
tweet_date         False
tweet_location      True
source             False
dtype: bool

In [16]:
missing_avg_temp = temp_df["AverageTemperature"].isnull().sum() / temp_df.shape[0]
missing_temp_uncertainity = temp_df["AverageTemperatureUncertainty"].isnull().sum() / temp_df.shape[0]

print(f"Ratio of missing values for average temperature {missing_avg_temp:.2f}")
print(f"Ratio of missing values for temperature uncertainty {missing_temp_uncertainity:.2f}")

Ratio of missing values for average temperature 0.04
Ratio of missing values for temperature uncertainty 0.04


In [17]:
print(temp_df.shape)
temp_df.dropna(inplace=True)

print(temp_df.shape)

(8599212, 7)
(8235082, 7)


In [18]:
missing_tweet_location = tweet_df["tweet_location"].isnull().sum() / tweet_df.shape[0]

print(f"Ratio of missing values for tweet location: {missing_tweet_location:.2f}")

Ratio of missing values for tweet location: 0.97


#### Duplicates

In [19]:
happiness_df.duplicated().any()

False

In [20]:
temp_df.duplicated().any()

False

In [21]:
tweet_df.duplicated().any()

False

#### Sneak peak into data

In [22]:
happiness_df.head()

,Country,Happiness.Rank,Happiness.Score,Whisker.high,Whisker.low,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715
3,Switzerland,4,7.494,7.561772,7.426227,1.564980,1.516912,0.858131,0.620071,0.290549,0.367007,2.276716
4,Finland,5,7.469,7.527542,7.410458,1.443572,1.540247,0.809158,0.617951,0.245483,0.382612,2.430182


In [23]:
happiness_df.tail()

,Country,Happiness.Rank,Happiness.Score,Whisker.high,Whisker.low,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual
150,Rwanda,151,3.471,3.543030,3.398970,0.368746,0.945707,0.326425,0.581844,0.252756,0.455220,0.540061
151,Syria,152,3.462,3.663669,3.260331,0.777153,0.396103,0.500533,0.081539,0.493664,0.151347,1.061574
152,Tanzania,153,3.349,3.461430,3.236570,0.511136,1.041990,0.364509,0.390018,0.354256,0.066035,0.621130
153,Burundi,154,2.905,3.074690,2.735310,0.091623,0.629794,0.151611,0.059901,0.204435,0.084148,1.683024
154,Central African Republic,155,2.693,2.864884,2.521116,0.000000,0.000000,0.018773,0.270842,0.280876,0.056565,2.066005


In [24]:
temp_df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
5,1744-04-01,5.788,3.624,Århus,Denmark,57.05N,10.33E
6,1744-05-01,10.644,1.283,Århus,Denmark,57.05N,10.33E
7,1744-06-01,14.051,1.347,Århus,Denmark,57.05N,10.33E
8,1744-07-01,16.082,1.396,Århus,Denmark,57.05N,10.33E


In [25]:
temp_df["dt"] = pd.to_datetime(temp_df["dt"], infer_datetime_format=True)

Summary stats for average temperatue by country and city recorded since 2000

In [26]:
temp_df[temp_df.dt.dt.year >= 2000].groupby([temp_df.Country, temp_df.City])["AverageTemperature"].describe()

count       mean       std     min       25%      50%  \
Country     City                                                               
Afghanistan Baglan     164.0  12.075945  9.483642  -7.700   4.19800  12.4715   
            Gardez     164.0  18.377762  8.156264   3.151  10.95200  19.4445   
            Gazni      164.0  11.557043  9.338659  -6.743   2.94700  12.1895   
            Herat      164.0  15.761195  9.353346  -6.633   7.76575  16.0660   
            Jalalabad  164.0  15.525756  8.640416  -0.906   8.66475  15.8495   
...                      ...        ...       ...     ...       ...      ...   
Zimbabwe    Gweru      164.0  20.220012  3.116030  13.674  17.72325  21.1105   
            Harare     164.0  20.721988  2.780533  14.623  18.57375  21.6565   
            Kadoma     164.0  20.721988  2.780533  14.623  18.57375  21.6565   
            Kwekwe     164.0  20.721988  2.780533  14.623  18.57375  21.6565   
            Mutare     164.0  19.112000  2.837385  12.838  16.69350  20.2970   

                            75%     max  
Country     City                         
Afghanistan Baglan     21.21600  25.261  
            Gardez     26.61225  29.246  
            Gazni      20.85750  24.600  
            Herat      24.70325  29.347  
            Jalalabad  24.30225  27.583  
...                         ...     ...  
Zimbabwe    Gweru      22.71250  25.508  
            Harare     22.78100  25.224  
            Kadoma     22.78100  25.224  
            Kwekwe     22.78100  25.224  
            Mutare     21.40100  22.927  

[3490 rows x 8 columns]

In [27]:
tweet_df.head()

,user_id,name,nickname,description,user_location,followers_count,tweets_count,user_date,verified,tweet_id,text,favs,retweets,tweet_date,tweet_location,source
0,3130604237,Sebugwaawo,njukihassan,Professional Counselor ~ Mentor ~ Cultural Koj...,Uganda,312,1763,2015-03-31 09:37:14,False,1164835451884060672,#FridayMotivation #FridayFeeling #FridaysForFu...,4,2,2019-08-23 09:43:07,None,Twitter Web App
1,876033974853931008,Alexander Reitzenstein,AlexReitzenst,Policy Advisor @E3G & Fellow @DPZ_Berlin | #Cl...,"Berlin, Deutschland",670,858,2017-06-17 11:09:10,False,1164835448071393280,Just to highlight: This could be a game change...,21,9,2019-08-23 09:43:07,None,Twitter Web App
2,1139443963914534912,Extinction Rebellion Bulgaria,XRBulgaria,,Bulgaria,194,19,2019-06-14 08:06:25,False,1164835271919095808,Today we are going to protest in front of the ...,43,10,2019-08-23 09:42:25,Bulgaria,Twitter for iPhone
3,904795229190017024,Kent Bäck,ocilari,🔍Kent Agent 🔎 #Constructivethoughts only. \n#S...,~ EARTH,809,9939,2017-09-04 19:56:07,False,1164834757659598848,#NewDays\n#oneday\n\nOne day!\n\nThis is the d...,2,0,2019-08-23 09:40:22,None,Twitter for Android
4,109275078,Sven Egenter,segenter,"Runs @cleanenergywire; @klimafakten; climate, ...",Berlin,1280,5086,2010-01-28 14:31:48,False,1164834415198867456,#Russia even has its own #fridaysforfuture mov...,4,4,2019-08-23 09:39:00,None,TweetDeck


User information of verified users

In [28]:
tweet_df.loc[(tweet_df["verified"] == True), ["nickname", "name", "description", "user_location", "source"]]

,nickname,name,description,user_location,source
116,AkashvaniAIR,ALL INDIA RADIO,"Official Account of All India Radio, \nIndia's...",India,Twitter Web Client
359,Emma_Gregg,Emma Gregg,"Travel journalist, editor, writer, photographe...",UK,Twitter Web App
410,mattgrocoff,Matt Grocoff,Called the 'Zero Energy Master'. Thrive Collab...,"Ann Arbor, MI, Earth",Twitter Web App
411,Raffi_RC,Raffi Cavoukian,Troubadour. writer. children's champion. ecolo...,Canada,Twitter for iPhone
414,Raffi_RC,Raffi Cavoukian,Troubadour. writer. children's champion. ecolo...,Canada,Twitter for iPhone
501,UWC_IO,UWC International,"UWC makes education a force to unite people, n...",Worldwide,Twitter Web App
535,CleanAirMoms,Moms Clean Air Force,We're a community of moms and dads who are uni...,United States,Twitter for Android
563,Fisher_DanaR,Dana R Fisher,Democracy: activism-protest-climate-the enviro...,"Maryland, USA",Twitter for Android
618,dw_environment,DW Global Ideas & Environment,"Reporting on solutions to the #climatecrisis, ...","Berlin, Germany",Twitter Ads Composer
637,Geeta_Mohan,Geeta Mohan گیتا موہن गीता मोहन,"Foreign Affairs Editor, @IndiaToday, TV Today ...","New Delhi, India",Twitter for Android


### Save and upload data to S3

In [24]:
temp_df.to_csv(data_path.joinpath("temperature_city_clean.csv"))

In [ ]:
files = ["happiness_index.csv", "temperature_city_clean.csv"]

for file in files:
    s3.upload_file(str(data_path.joinpath(file)), config.get("kinesis", "delivery_stream"), file)